In [1]:
#!pip install requests

In [2]:
import requests
import datetime
from urllib.parse import urlencode
import json
import spotipy
import spotipy.util as util

In [3]:
import base64

In [4]:
client_id = '9ec14c18b8114a569770be752b524b7e'
client_secret = '6d13d61e65a2418697e225936274c7bf'

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    auth_token= "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_cred = f"{client_id}:{client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64.decode()
        
    def get_token_header(self):
        client_cred_b64 = self.get_client_credentials()
        return{
             "Authorization" : f"Basic {client_cred_b64}"
        }
    def get_token_data(self):
        return{
            "grant_type": "client_credentials"
        }
    
    def Authorization(self):
        auth_token = self.auth_token
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(auth_token,data=token_data, headers=token_header)
        if r.status_code not in range(200,299):
            raise Exception("could not authenticate client")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.Authorization()
            return self.get_access_token()
        elif token == None:
            return self.get_access_token()
        return token
    
    def get_resources_header(self):
        access_token = self.get_access_token()
        header = {
            "Authorization": f"Bearer {access_token}"
        }
        return header
    
    def get_resources(self, _id, resource_type="album", version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{_id}"
        header = self.get_resources_header()
        r = requests.get(endpoint, headers=header)
        if r.status_code not in range(200,299):
            return{}
        return r.json()
    
    def get_album(self,_id):
        return self.get_resources(_id,resource_type='albums')
    
    def get_artist(self,_id):
        return self.get_resources(_id,resource_type='artists')
    
    def get_artist_albums(self,_id):
        return self.get_resources(_id,resource_type='artist_albums')

    def get_track(self,_id):
        return self.get_resources(_id,resource_type='tracks')
    
    
    def base_search(self, query_parameters, search_type='artists'):
        access_token = self.get_access_token()
        header = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_parameters}"
        r = requests.get(lookup_url, headers=header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
        
    def search(self,query= None, operator=None, operator_query=None,search_type='artist'):
        if query == None:
            raise Exception('Query Required')
        if isinstance(query,dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator !=None and operator_query !=None:
            if operator.lower() =="or" or operator.lower() =="not":
                operator = operator.upper()
            if isinstance(operator_query,str):
                query= f"{query}{operator}{operator_query}"
        query_parameters = urlencode({"q": query, "type": search_type.lower()})
        print(query_parameters)
        return self.base_search(query_parameters)
        
        

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
#finding artist
searchQuery = input("Search: ")
#print()
searchResult = spotify.search(searchQuery,"artist")
print(json.dumps(searchResult, sort_keys=True, indent=4))


q=pink+flyod&type=artist
{
    "artists": {
        "href": "https://api.spotify.com/v1/search?query=pink+flyod&type=artist&offset=0&limit=20",
        "items": [
            {
                "external_urls": {
                    "spotify": "https://open.spotify.com/artist/0k17h0D3J5VfsdmQ1iZtE9"
                },
                "followers": {
                    "href": null,
                    "total": 15587170
                },
                "genres": [
                    "album rock",
                    "art rock",
                    "classic rock",
                    "progressive rock",
                    "psychedelic rock",
                    "rock",
                    "symphonic rock"
                ],
                "href": "https://api.spotify.com/v1/artists/0k17h0D3J5VfsdmQ1iZtE9",
                "id": "0k17h0D3J5VfsdmQ1iZtE9",
                "images": [
                    {
                        "height": 977,
                        "url": "https://i.s

In [8]:
#artist details
artist = searchResult['artists']['items'][0]
print("Artist Name: " + (artist['name']))
print("Artist Follower: " + (str(artist['followers']['total']) + " followers"))
print("Artist Genres: " + (str(artist['genres'])))
print()
artistID = artist['id']
#print(artistID)
#webbrowser.open(artist['images'][0]['url'])

"""
album and tracks of artist
trackURIs = []
trackArt = []
"""
i = 0

#Scraping album info


albumResult = spotify.search(artistID)
print(json.dumps(albumResult, sort_keys=True, indent=4))


#albumResult = albumResult['items']

"""
for item in albumResult:
    print("Album" + item['name'])
    albumID = item['id']
    #albumArt = item['images'][0]['uri]

    #Albums have tracks[scraping track data]
    trackResult = spotify.get_track(albumID)
    trackResult = trackResult['items']

    for item in trackResult:
        print(str(i) + ":" + item['name'])
        trackURIs.append(item['uri'])
        #trackArt.append(albumArt)
        i+=1
    print()
"""

Artist Name: Pink Floyd
Artist Follower: 15587170 followers
Artist Genres: ['album rock', 'art rock', 'classic rock', 'progressive rock', 'psychedelic rock', 'rock', 'symphonic rock']

q=0k17h0D3J5VfsdmQ1iZtE9&type=artist
{
    "artists": {
        "href": "https://api.spotify.com/v1/search?query=0k17h0D3J5VfsdmQ1iZtE9&type=artist&offset=0&limit=20",
        "items": [],
        "limit": 20,
        "next": null,
        "offset": 0,
        "previous": null,
        "total": 0
    }
}


'\nfor item in albumResult:\n    print("Album" + item[\'name\'])\n    albumID = item[\'id\']\n    #albumArt = item[\'images\'][0][\'uri]\n\n    #Albums have tracks[scraping track data]\n    trackResult = spotify.get_track(albumID)\n    trackResult = trackResult[\'items\']\n\n    for item in trackResult:\n        print(str(i) + ":" + item[\'name\'])\n        trackURIs.append(item[\'uri\'])\n        #trackArt.append(albumArt)\n        i+=1\n    print()\n'

In [9]:
spotify.search({"album": "Ye","artist":"Kanye"}, search_type="track")

q=album%3AYe+artist%3AKanye&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=album%3AYe+artist%3AKanye&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
       'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
       'id': '5K4W6rqBFWDnAN6FQUkS6x',
       'name': 'Kanye West',
       'type': 'artist',
       'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

In [10]:
#spotify.get_artist("5K4W6rqBFWDnAN6FQUkS6x")

In [11]:
spotify.get_album("4UlGauD7ROb3YbVOFMgW5u")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
   'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
   'id': '5pKCCKE2ajJHZ9KAiaK11H',
   'name': 'Rihanna',
   'type': 'artist',
   'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH',
  

In [12]:
#spotify.search(query="Work", operator='NOT', operator_query='it',search_type="track")